In [1]:
import sys
 
sys.path.append('..')
from piece import piece
piece.start()

In [2]:
piece.reset()
piece.synth_player.start()
piece.synth_player.note() \
    .sine(piece.control_instruments.static_control(440), piece.control_instruments.sine_control(0, 1)) \
    .pan(piece.control_instruments.static_control(0)) \
    .play(0, 1)

piece.synth_player.note() \
    .sine(piece.control_instruments.static_control(220), piece.control_instruments.sine_control(0, 1)) \
    .pan(piece.control_instruments.static_control(0)) \
    .play(1, 1)
    


In [2]:
piece.reset()
piece.synth_player \
    .add_sound("wood_hit_1", "/Users/danielstahl/Documents/Projects/py-soundmining-tools/samples/Wood hit 1.flac", 0.201, 0.339) \
    .start()

In [5]:
piece.synth_player.note() \
    .sound_mono("wood_hit_1", 1, piece.control_instruments.static_control(1)) \
    .pan(piece.control_instruments.static_control(0)) \
    .play(0)

In [6]:
piece.stop()

In [ ]:
from math import pi

from ipycanvas import Canvas

canvas = Canvas(width=200, height=200)

canvas.fill_style = "red"
canvas.stroke_style = "blue"

canvas.fill_arc(60, 60, 50, 0, pi)
canvas.stroke_circle(60, 60, 40)

canvas